## A notebook to create animated, exportable timelapse videos, like this one:

<video width="400" height="400" 
       src="/Example_footage/final2.gif"  
       controls>
</video>

We will heavily rely on "geemap". Geemap is a Python package for interactive geospatial analysis and visualization with Google Earth Engine (GEE), created an maintained by Qiusheng Wu. 
Many tutorials, notebooks and extensive docs can be found on [www.geemap.org](https://geemap.org) and the related YouTube channel ["Open Geospatial Solutions"](https://www.youtube.com/@giswqs).

For the original GEE docs, check https://developers.google.com/earth-engine/apidocs, where more and more code snippets are now also shown in the Python language.

Requirement for this notebook to work: you must have an account with [GEE](https://earthengine.google.com/noncommercial/).

### Step 0: Install packages

Give needed packages as a requirements.txt file

### Step 1: Import relevant packages. When running this notebook for the first time, uncomment ee.Authenticate and ee.Initialize to authenticate your GEE profile.

In [ ]:
import ee
import geemap
import os
import ipyleaflet
import ipywidgets as widgets
from pygifsicle import gifsicle
import math
import geopy.distance
#ee.Authenticate()
#ee.Initialize()
Map = geemap.Map()

: 

In [21]:
# import session_info
# session_info.show()

### Step 2: Produces an interactive map with timeline-hotspots in a dropdown. You can also just draw your own polygon, or explore the other build in tools.

In [22]:
dropdown = widgets.widgets.Dropdown(
    options={'Wyoming': [[[-105.597746, 43.463884], [-105.597746, 43.770102], [-104.894357, 43.770102], [-104.894357, 43.463884], [-105.597746, 43.463884]]], 
             'Dubai': [[[54.881176, 24.95618], [54.881176, 25.262085], [55.493822, 25.262085], [55.493822, 24.95618], [54.881176, 24.95618]]], 
             'Rondonia': [[[-64.62588, -10.669605], [-62.04939934049605, -10.669605], [-62.04939934049605, -9.241285], [-64.62588, -9.241285], [-64.62588, -10.669605]]],
             'Alberta': [[[-112.088105, 56.848972], [-112.088105, 57.396144], [-110.395576, 57.396144], [-110.395576, 56.848972], [-112.088105, 56.848972]]],
             'Saudi Arabia': [[[37.976156, 29.812845], [37.976156, 30.411571], [39.064468, 30.411571], [39.064468, 29.812845], [37.976156, 29.812845]]],
             'Las Vegas': [[[-115.351475, 35.973561], [-115.351475, 36.259778], [-114.759224, 36.259778], [-114.759224, 35.973561], [-115.351475, 35.973561]]],
             'Dar es Salam': [[[39.155018, -6.888907], [39.396055841047456, -6.888907], [39.396055841047456, -6.753914], [39.155018, -6.753914], [39.155018, -6.888907]]],
             'Donguan': [[[113.562346, 22.815428], [113.562346, 23.15425], [114.166417, 23.15425], [114.166417, 22.815428], [113.562346, 22.815428]]],
             'Delta, Spain': [[[0.762055, 40.64827], [1.0150245872566783, 40.64827], [1.0150245872566783, 40.756544], [0.762055, 40.756544], [0.762055, 40.64827]]],
             'Playa del Carmen': [[[-87.20761099999999, 20.550415], [-86.94968016182884, 20.550415], [-86.94968016182884, 20.68666], [-87.20761099999999, 20.68666], [-87.20761099999999, 20.550415]]],
             'Atchafalaya': [[[-91.56306899999998, 29.376902], [-91.19337290616403, 29.376902], [-91.19337290616403, 29.558638], [-91.56306899999998, 29.558638], [-91.56306899999998, 29.376902]]],
             'Bhadla Solar Park': [[[71.886475, 27.446293], [72.08417949906001, 27.446293], [72.08417949906001, 27.545269], [71.886475, 27.545269], [71.886475, 27.446293]]]
            }, 
            description='Hotspots:'
)

#output_widget.clear_output() #uncomment this line in case you run this cell NOT for the first time
output_widget = widgets.Output(layout={'border': '5px dashed green'})
output_control = ipyleaflet.WidgetControl(widget=output_widget, position='topright')
Map.add_control(output_control)

with output_widget:
    display(dropdown)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Step 3: Choose one of three options to create your 'geom', comment the two options you don't use.
#### Important: each time you choose another hotspot from the dropdown menu, you have to run this cell again, to update your 'geom' variable

In [23]:
geom = ee.Geometry.Polygon(dropdown.value)
#geom = ee.Geometry.Polygon([[[-106.94217152931594, 26.214060591974505], [-106.92224071182336, 26.214060591974505], [-106.92224071182336, 26.232054865951508], [-106.94217152931594, 26.232054865951508], [-106.94217152931594, 26.214060591974505]]])
#geom = Map.user_roi

Map.addLayer(geom, {'color': 'FFFFFF'}, 'geodesic polygon')
Map.centerObject(geom, 12) #the number defines the zoom level
Map

Map(bottom=812.0, center=[43.61740211080997, -105.24605150000028], controls=(WidgetControl(options=['position'…

### Step 4: Choose the datasets to be used. This example uses Landsat 5, 7, 8 and 9 and merges them into one. For merging different sensors, for example Landsat with Sentinel, see notebook "MergingTimelapse".

Info regarding the Landsat collections:

Start date L5: 1984/3/16
End date L5: 2012/5/5
                             
Start date L7: 1999/5/28
End date L7: 2022/4/6
Important: scan line correction problem since 31.05.2003, so we rather do not use Landsat 7. However, there are regions where Landsat 5 & 8 coverage is failing or the input is just very low, for example, often in the year 2012. This also depends on the region. Use only the necessary years of L7! If several non-consecutive periods, merge them into one.

Start date L8: 2013/3/18
End date L8: still active

Start date L9: 2021/10/31
End date L9: still active

In [24]:
# if using Landsat 7, choose your start and end date first

start_dateL7 = ee.Date.fromYMD(1999, 1, 1)
end_dateL7 = ee.Date.fromYMD(2012, 12, 31)

landsat5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
landsat7_sel = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterDate(start_dateL7, end_dateL7)
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

# rename the bands, so red is simply red for all collections

L5_bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
L7_bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
L8_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
L9_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']

bandNamesLandsat = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']

landsat5renamed = landsat5.select(L5_bands, bandNamesLandsat)
landsat7renamed = landsat7_sel.select(L7_bands, bandNamesLandsat)
landsat8renamed = landsat8.select(L8_bands, bandNamesLandsat)
landsat9renamed = landsat9.select(L9_bands, bandNamesLandsat)

# merge different collections

l57 = landsat5renamed.merge(landsat7renamed)
l578 = l57.merge(landsat8renamed)
l5789 = l578.merge(landsat9renamed)

l58 = landsat5renamed.merge(landsat8renamed)
l589 = l58.merge(landsat9renamed)

### Step 5: Decide for your timelapse if you want (a mosaic of the) best image of each period, or maybe a median or mean image over this period.

In [25]:
# for the best image per year:

def get_best_image(year):
    start_date = ee.Date.fromYMD(year, 1, 1) # set your start month and day
    end_date = ee.Date.fromYMD(year, 3, 31) # set your end month and day
    image = (
        ee.ImageCollection(l5789) # enter your preferred (merged) collection here
        .filterBounds(geom) 
        .filter(ee.Filter.contains('.geo', geom)) # filter to make sure that your whole polygon is filled with imagery
        .filterDate(start_date, end_date) 
        .filter(ee.Filter.gte('CLOUD_COVER_LAND', 0)) # filter out images without cloud cover score
        .sort('CLOUD_COVER_LAND', False) # sort the collection by inversed cloud cover, meaning your mosaic will be build with the images with least clouds
        .mosaic()
        .clip(geom)
        .set('year', year) # add the 'year' property to your processed image
    )
    return ee.Image(image)

In [26]:
# for a median image per year:

def get_median_image(year):
    start_date = ee.Date.fromYMD(year, 1, 1) # set your start month and day
    end_date = start_date.advance(12, 'month') # set your 'advance' period, to reach your end date

    image = (
        ee.ImageCollection(l5789) # enter your preferred (merged) collection here
        .filterBounds(geom) 
        .filterDate(start_date, end_date) 
        .filter(ee.Filter.contains('.geo', geom))  # filter to make sure that your whole polygon is filled with imagery
        .filter(ee.Filter.gte('CLOUD_COVER_LAND', 0)) # filter out images without cloud cover score
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', 10) # set your maximum allowed cloud cover (%)
        .median() # there is more options, like mean (see Documentation)
        .clip(geom)
        .set('year', year) # add the 'year' property to your processed image
    )
    return image

### Step 6: Choose your start and end years for the timelapse and run them over your preferred function: get_median_image or get_best_image

In [27]:
start_year = 1990
end_year = 2023

years_collection = ee.List.sequence(start_year, end_year)

collection = ee.ImageCollection(years_collection.map(get_median_image)) # enter your chosen function here
collection #shows the size of your collection, including all properties

### Step 7: Choose your visualisation parameters from the premade ones or pick your own values.
#### Optional: from the layer selector on the map, you can choose the 'settings symbol' per layer and manipulate the visualisation parameters. Play around and if happy, import the new values to your cell.

In [28]:
# vis_params = {'bands': ['red', 'green', 'blue'], 'min': 8047, 'max': 13635, 'opacity': 1.0, 'gamma': 1.0} #Delta
# vis_params = {'bands': ['red', 'green', 'blue'], 'min': 939, 'max': 20405, 'opacity': 1.0, 'gamma': 0.76} #Las Vegas
vis_params = {'bands': ['red', 'green', 'blue'], 'min': 6559, 'max': 22455, 'opacity': 1.0, 'gamma': 1.0} #Dubai
# vis_params = {'bands': ['red', 'green', 'blue'], 'min': 7001, 'max': 12001, 'opacity': 1.0, 'gamma': 1.0} # Rondonia
# vis_params = {'bands': ['red', 'green', 'blue'], 'min': 7356, 'max': 16250, 'opacity': 1.0, 'gamma': 1.0} # Wyoming
# vis_params = {'bands': ['red', 'green', 'blue'], 'min': 7000, 'max': 16669, 'opacity': 1.0, 'gamma': 1.0} #Alberta
# vis_params = {'bands': ['nir', 'red', 'green'], 'min': 12930, 'max': 26499, 'opacity': 1.0, 'gamma': 1.0} # Saudi
# vis_params = {'bands': ['swir2', 'nir', 'green'], 'min': 2639, 'max': 17405, 'opacity': 1.0, 'gamma': 0.76} # Donguan
# vis_params = {'bands': ['red', 'green', 'blue'], 'min': 800, 'max': 23170, 'opacity': 1.0, 'gamma': 1.0} # Bhadla

# let's visualize the first image of the collection
image = ee.Image(collection.first())
Map.addLayer(image, vis_params, 'First Landsat image')
Map

Map(bottom=383127.0, center=[43.61740211080997, -105.24605150000028], controls=(WidgetControl(options=['positi…

In [29]:
# when satisfied with your vis_params, force them on all images in your collection

def visuals(img):
  return img.visualize(**vis_params).copyProperties(img, img.propertyNames())

coll_preview = collection.map(visuals)
coll_preview

### Step 8: Create a preview of your timelapse with the image collection we created so far

In [81]:
# this cell produces a link to a filmstrip, which you can open in another tab. It allows for a sneak peek on all images in your collection.
# The indexed list makes it easier to see which years are not optimal

video_args_small = {
    'dimensions': [500, 281], # width * height, keep this value low for now
    'framesPerSecond': 2
}

print(coll_preview.getFilmstripThumbURL(video_args_small))

# for an indexed list of years in your collection
coll_year_preview = coll_preview.aggregate_array('year').getInfo()
for i,year in enumerate(coll_year_preview, start=1):
    print (str(i) + '.', year)

https://earthengine.googleapis.com/v1/projects/earthengine-legacy/filmstripThumbnails/0b2725555b5665821f58ec778f71e3db-9564960fa0da63885c6b99993018f086:getPixels
1. 1990
2. 1991
3. 1992
4. 1993
5. 1994
6. 1995
7. 1996
8. 1997
9. 1998
10. 1999
11. 2000
12. 2001
13. 2002
14. 2003
15. 2004
16. 2005
17. 2006
18. 2007
19. 2008
20. 2009
21. 2010
22. 2011
23. 2012
24. 2013
25. 2014
26. 2015
27. 2016
28. 2017
29. 2018
30. 2019
31. 2020
32. 2021
33. 2022
34. 2023


In [82]:
# this cell produces, stores and shows a small gif of your collection

saved_gif = 'preview.gif' # give a name to your preview gif
geemap.download_ee_video(coll_preview, video_args_small, saved_gif) 
geemap.show_image(saved_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/jochem/preview.gif


Output()

In [34]:
# remove years with no or sub-optimal imagery, by uncommenting the lines below and choosing the years to remove

coll_selected = (
    coll_preview
    .filter(ee.Filter.neq('year', 2012))
    # .filter(ee.Filter.neq('year', 2013))
    # .filter(ee.Filter.neq('year', 2014))
)

### Step 9: In order to create a HD-timelapse (1920*1080 pixels), we need to break up our collection in parts of 12 images each (otherwise we quickly reach the max capacity of GEE)

In [35]:
coll_year_list = coll_selected.aggregate_array('year').getInfo()

pt1 = coll_year_list[:12]
pt2 = coll_year_list[12:24]
pt3 = coll_year_list[24:36]

pt1gif = coll_selected.filter(ee.Filter.inList('year', pt1))
pt2gif = coll_selected.filter(ee.Filter.inList('year', pt2))
pt3gif = coll_selected.filter(ee.Filter.inList('year', pt3))

print(coll_selected.getFilmstripThumbURL(video_args_small)) # a new filmstrip

for i,year in enumerate(coll_year_list, start=1):
    print (str(i) + '.', year) # an indexed list of years in your collection

https://earthengine.googleapis.com/v1/projects/earthengine-legacy/filmstripThumbnails/efce0a30b9df9e47c87889d094a551f0-10fbc6ce0c2156d4e52a159dd097c9ad:getPixels
1. 1990
2. 1991
3. 1992
4. 1993
5. 1994
6. 1995
7. 1996
8. 1997
9. 1998
10. 1999
11. 2000
12. 2001
13. 2002
14. 2003
15. 2004
16. 2005
17. 2006
18. 2007
19. 2008
20. 2009
21. 2010
22. 2011
23. 2013
24. 2014
25. 2015
26. 2016
27. 2017
28. 2018
29. 2019
30. 2020
31. 2021
32. 2022
33. 2023


### Step 10: Let's produce that complete HD Timelapse by creating the parts and stitching them together

In [36]:
video_args = {
    'dimensions': [1920,1080],
    'framesPerSecond': 2
}

print(pt1gif.getFilmstripThumbURL(video_args))
print(pt2gif.getFilmstripThumbURL(video_args))
print(pt3gif.getFilmstripThumbURL(video_args))

https://earthengine.googleapis.com/v1/projects/earthengine-legacy/filmstripThumbnails/7c9fc8b362009ebf9384299706600ec3-b9228a1cc93591290251056689ae95e6:getPixels
https://earthengine.googleapis.com/v1/projects/earthengine-legacy/filmstripThumbnails/1e23ee5abda9889c42fd8c751d8ec7b4-408b54e39321077e4f9628af3c46051e:getPixels
https://earthengine.googleapis.com/v1/projects/earthengine-legacy/filmstripThumbnails/1acca38edac56cd0c8ff3e7bd9e95028-8b654192ef236ce15720fd014f77d28b:getPixels


In [37]:
saved_gif1 = 'pt1.gif'
saved_gif2 = 'pt2.gif'
saved_gif3 = 'pt3.gif'

geemap.download_ee_video(pt1gif, video_args, saved_gif1)
geemap.download_ee_video(pt2gif, video_args, saved_gif2)
geemap.download_ee_video(pt3gif, video_args, saved_gif3)

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/jochem/pt1.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /Users/jochem/pt2.gif
Generating URL...
Please wait ...
The GIF image has been saved to: /Users/jochem/pt3.gif


In [38]:
out_gif = 'HD_no_animation.gif' # name your complete, non-animated HD gif
geemap.merge_gifs(['pt1.gif', 'pt2.gif', 'pt3.gif'], out_gif)

### Step 11: We can animate our gif with years, title, scalebar, etc.

In [80]:
# this calculates the size for the scalebar
coordinates = geom.coordinates().flatten()
coordinates2 = coordinates.getInfo()
x1 = coordinates2[0]
y1 = coordinates2[1]
y2 = coordinates2[5]
x2 = coordinates2[2]

coords_1 = [y1,x1]
coords_2 = [y1,x2]
width = geopy.distance.geodesic(coords_1, coords_2).km
width_geom_km = round(width, 2)
print(width_geom_km, 'km')

# when creating HD timelapses, the screen width is 1920 pixels. I like a scalebar that is ca. 1/10 of the 1920 pixels.
# For example: for Wyoming, with a geo_width_km of 56.92, I chose a 5km scalebar.

pixels_per_km = 1920/width_geom_km
chosen_km_scalebar = 5 #choose your km

length_scalebar = round(pixels_per_km * chosen_km_scalebar)
height_scalebar = round((length_scalebar / 3.1837)) #this 3.1837 is fixed. It is derived from the scalebar png that I made.

56.92 km


In [78]:
yearstamp = [str(n) for n in (coll_year_list)]

out_gif = 'HD_animated.gif' # name your animated HD gif

# add year label

geemap.add_text_to_gif(
    'HD_no_animation.gif',
    out_gif,
    xy=('90%', '5%'), # placement of this label
    text_sequence= yearstamp,
    font_size=50,
    font_type="Arial.ttf",
    font_color='white',
    add_progress_bar=False
)

# add scalebar, choose which color of scalebar you want and set the right text ('5 km', '10 km', etc)

scalebar = 'scalebar_black.png'

geemap.add_image_to_gif(
    out_gif,
    out_gif,
    in_image=scalebar,
    xy=('88%', '90%'),
    image_size=(length_scalebar, height_scalebar),
    circle_mask=False
)

geemap.add_text_to_gif(
    out_gif,
    out_gif,
    xy=('90.5%', '87%'),
    text_sequence= (str(chosen_km_scalebar) + ' km'),
    font_size=30,
    font_color='white',
    add_progress_bar=False
)

# add title

geemap.add_text_to_gif(
    out_gif,
    out_gif,
    xy=('3%', '5%'),
    text_sequence='Dubai, United Arab Emirates',
    font_size=50,
    font_color='white',
    add_progress_bar=False
)

# add subtitle and choose frame duration

geemap.add_text_to_gif(
    out_gif,
    out_gif,
    xy=('3%', '10%'),
    text_sequence='Urbanization',
    duration=1000, # duration per frame in milliseconds
    font_size=30,
    font_color='white',
    add_progress_bar=False
)

geemap.show_image(out_gif)

Output()

In [71]:
# export gif as mp4 to your current or chosen folder

out_mp4 = 'My_first_awesome_TL'
geemap.gif_to_mp4(out_gif, out_mp4)

In [ ]:
# export all images as PNG to your current or chosen folder

out_dir = os.path.join(os.path.expanduser('~'), 'Timelapse/All_images_of_my_first_awesome_TL')
geemap.gif_to_png(out_gif, out_dir, prefix='', verbose=True)